### Step-1: Required Imports

In [ ]:
from transformers import pipeline

### Step-2: Device Setup

This function `get_device()` checks and selects the most suitable device for model computation using PyTorch. It supports CUDA GPUs, Intel XPUs, and falls back to the CPU if no accelerator is available.

- **torch.cuda.is_available()**: Checks for NVIDIA GPUs with CUDA support.
- **torch.xpu.is_available()**: Checks for Intel XPUs.
- **torch.device()**: Creates a device object representing the selected device (CPU, CUDA, or XPU).
- **torch.xpu.empty_cache()**: Clears the memory cache for XPUs to optimize memory usage before running a model.

In [4]:
def get_device() -> torch.device:
    """Check and return the appropriate device (XPU, CUDA, or CPU)."""
    if torch.cuda.is_available():
        device_type = "cuda"
        device = torch.device(device_type)
        print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
    elif torch.xpu.is_available():
        device_type = "xpu"
        device = torch.device(device_type)
        torch.xpu.empty_cache()  # Empty the XPU cache if using XPU
        print(f"Using device: {torch.xpu.get_device_name()}")
    else:
        device_type = "cpu"
        device = torch.device(device_type)
        print("Using CPU")
        
    return device

### Step 3: Prepare the Text for Summarization

In [6]:
article = """
Throughout history, the rise and fall of ancient civilizations have been marked by extraordinary achievements, wars, and sometimes, a gradual decline due to internal or external forces. One of the earliest known civilizations emerged in the ancient Near East around 3,500 BCE: the Sumerians. Their city-states, such as Ur, Uruk, and Eridu, flourished along the Tigris and Euphrates Rivers in modern-day Iraq. The Sumerians are credited with many advancements, including the invention of cuneiform writing, which laid the foundation for record-keeping, literature, and law. They also developed sophisticated systems of irrigation, enabling them to thrive in a region that would otherwise have been inhospitable.

However, like many ancient civilizations, the Sumerians faced constant threats from outside invaders and internal strife. Over time, their cities fell to powerful empires, including the Akkadians and Babylonians. The Akkadian Empire, established by Sargon of Akkad around 2,330 BCE, is often considered the first ancient empire in history. Sargon’s conquests led to the spread of Akkadian culture and the unification of the region under a centralized government. But the Akkadian Empire eventually collapsed due to a combination of factors, including prolonged droughts, invasions, and internal political instability.

The next major civilization to dominate the region was the Babylonians. Under the rule of King Hammurabi (1792-1750 BCE), Babylon became the center of power in the ancient world. Hammurabi is famous for codifying the laws of his empire in what is now known as the Code of Hammurabi, one of the oldest deciphered writings of significant length in the world. The Babylonian Empire, however, eventually fell to the Persians in 539 BCE after a long period of internal conflict and external pressures from neighboring nations.

Meanwhile, on the other side of the world, the ancient Egyptians were constructing one of the most enduring civilizations in history. The early dynastic period of Egypt saw the establishment of the first centralized state around 3,100 BCE under the leadership of King Narmer. The civilization’s longevity and stability were driven by its ability to control the Nile River, which provided water for agriculture and transportation. Over the centuries, Egypt saw the construction of iconic monuments, including the pyramids of Giza, the Sphinx, and massive temples dedicated to their gods and pharaohs.

The Egyptian Empire reached its zenith during the New Kingdom period (1550-1070 BCE), when powerful pharaohs such as Ramses II and Thutmose III expanded Egypt’s borders and secured its influence in the ancient world. However, by the end of the New Kingdom, Egypt began to experience a slow decline. External invasions from the Sea Peoples, combined with internal social unrest, contributed to the weakening of the state. Eventually, Egypt was conquered by the Assyrians, and later the Persians, before falling to Alexander the Great in 332 BCE. After Alexander’s death, Egypt became part of the Hellenistic Ptolemaic Kingdom, which eventually faded after the reign of Cleopatra VII and the Roman conquest in 30 BCE.

Further east, the ancient Chinese civilization was developing its own rich history, marked by dynastic rule that spanned millennia. The earliest known Chinese dynasty, the Xia, dates back to around 2100 BCE, though much of its early history remains a matter of legend. The Shang Dynasty (1600-1046 BCE) is one of the first to be well-documented, with evidence of written records, advanced metallurgy, and complex religious rituals. The Zhou Dynasty, which followed the Shang, introduced the concept of the "Mandate of Heaven," which justified the ruler’s divine right to govern.

The Zhou Dynasty saw a period of expansion and philosophical development, with Confucius and Laozi offering ideas that would shape Chinese thought for centuries. Despite its cultural contributions, the Zhou Dynasty eventually fragmented into warring states, leading to the rise of the Qin Dynasty in 221 BCE. The Qin Emperor, Shi Huangdi, is famous for unifying China, standardizing weights and measures, and building the first version of the Great Wall. However, the Qin Dynasty was short-lived, with the emperor’s harsh rule leading to widespread rebellion shortly after his death.

The Han Dynasty (206 BCE-220 CE) succeeded the Qin and is often considered a golden age in Chinese history. Under the Han, China expanded its territory, established the Silk Road trade routes, and saw significant advancements in science, technology, and culture. But by the third century CE, internal corruption, external invasions, and the disintegration of central power led to the downfall of the Han Dynasty.

Similar patterns of rise and fall were witnessed in other regions, such as the ancient civilizations of Greece and Rome. The Greek city-states, most famously Athens and Sparta, contributed greatly to the fields of philosophy, art, and democracy. However, after the conquests of Alexander the Great, Greece fell under the control of the Macedonian Empire, which eventually fragmented. Rome, initially a republic, grew into one of the most powerful empires the world has ever seen. The Roman Empire reached its peak under Emperor Trajan (98-117 CE) but eventually declined due to internal decay, economic troubles, and invasions by barbarian tribes. The Western Roman Empire fell in 476 CE, while the Eastern Roman Empire, also known as the Byzantine Empire, continued for another thousand years.

In summary, the history of ancient civilizations is a complex tapestry of achievements, conflicts, and declines. Civilizations rose to power due to their ability to innovate, adapt, and conquer, but they often fell because of political instability, economic challenges, and military defeats. Despite their eventual decline, the legacies of these civilizations continue to influence the world today.
"""

In [ ]:
article

### Step-4: Pipeline creation

This code initializes a summarization pipeline using the Hugging Face `pipeline()` function.

- **"summarization"**: Specifies that the task is text summarization, where the model will generate a concise summary of the given text.
- **model="facebook/bart-large-cnn"**: Loads the pre-trained **BART-Large-CNN** model, which is specifically fine-tuned for summarization tasks.
- **device=get_device()**: Uses the `get_device()` function to select the most suitable device (CPU, GPU using CUDA, or Intel XPU) for running the model.

In [ ]:
# Load the summarizer pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=get_device())

### Step-5: Pipeline Initialization

This code uses the initialized summarization pipeline to generate a summary of the input text (`article`).

- **summary**: Stores the generated summary as the output of the `summarizer()` function.
- **article**: The input text that needs to be summarized.
- **max_length=400**: Specifies the maximum number of tokens allowed in the generated summary.
- **min_length=250**: Ensures the summary contains at least 250 tokens.
- **do_sample=False**: Disables sampling, making the model choose the most likely tokens for a more deterministic and accurate summary.

In [17]:
summary = summarizer(article, max_length=400, min_length=250, do_sample=False)

In [18]:
summary

[{'summary_text': 'The rise and fall of ancient civilizations have been marked by extraordinary achievements, wars, and sometimes, a gradual decline. One of the earliest known civilizations emerged in the ancient Near East around 3,500 BCE: the Sumerians. Over time, their cities fell to powerful empires, including the Akkadians and Babylonians. The earliest known Chinese dynasty, the Xia, dates back to around 2100 BCE, though much of its early history remains a matter of legend. The Zhou Dynasty introduced the concept of the "Mandate of Heaven," which justified the ruler’s divine right to govern. However, the Qin Dynasty is famous for unifying China, building the world\'s first version of the Great Wall of China. The Roman Empire was short-lived, with the emperor\'s rule under his death leading to widespread barbarian invasions. The fall of the Roman Empire led to the rise of theocracies such as the Roman Republic, Roman Empire, and Byzantine Empire. The downfall of Roman Empire saw th